[View in Colaboratory](https://colab.research.google.com/github/itsSKG/DrAIvex/blob/master/Project2.ipynb)

In [47]:
! pip install pydrive
# these classes allow you to request the Google drive API
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
file_id_train = '106NKNFoaR4RM690elN6_zuLWubpba4TQ'
file_id_testX = '1feGQO_rsRaplh9thTMBPESWtHvBLymHL'
file_id_testY = '1hap9oraPBrvRTuh9FgdUJ0rSEuZsnIJ2'
downloaded_train = drive.CreateFile({'id': file_id_train})
downloaded_testX = drive.CreateFile({'id': file_id_testX})
downloaded_testY = drive.CreateFile({'id': file_id_testY})

# allows you to temporarily load your file in the notebook VM

# assume the file is called file.csv and it's located at the root of your drive
downloaded_train.GetContentFile('file_train.csv')
downloaded_testX.GetContentFile('file_testX.csv')
downloaded_testY.GetContentFile('file_testY.csv')


# https://drive.google.com/file/d/106NKNFoaR4RM690elN6_zuLWubpba4TQ/view?usp=sharing --> train
# https://drive.google.com/file/d/1feGQO_rsRaplh9thTMBPESWtHvBLymHL/view?usp=sharing --> test_X
# https://drive.google.com/file/d/1hap9oraPBrvRTuh9FgdUJ0rSEuZsnIJ2/view?usp=sharing --> test_Y

In [0]:
train = pd.read_csv('file_train.csv')
testX = pd.read_csv('file_testX.csv')
testY = pd.read_csv('file_testY.csv')

In [46]:
testY.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
